<a href="https://colab.research.google.com/github/RaulMrSouza/Relatorio-Covid-por-Estado/blob/master/Relatorio_Covid_por_Estado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pacotes e imports iniciais
!pip install folium==0.8.2
import folium
from folium import plugins
import pandas as pd
import json
!pip install requests==v2.21.0
import requests
import io
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')

     |████████████████████████████████| 92kB 2.2MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.2 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.21.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.2 which is incompatible.
  Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
#@title Usar Runtime->Run (Ctrl+F10) after para atualizar relatório { display-mode: "form", run: "auto"  }

Estado = 'SP' #@param ["AC", "AL", "AM", "AP", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RO", "RS", "RR", "SC", "SE", "SP", "TO"]

top_10_cidades_em = "Taxa de Mortalidade" #@param [ "Casos Confirmados por 100k habitantes", "Total de Confirmados",  "Total de Obitos", "Taxa de Mortalidade"]
Mapa_coropletico_por = "Casos Confirmados por 100k habitantes" #@param [ "Casos Confirmados por 100k habitantes", "Total de Confirmados",  "Total de Óbitos", "Taxa de Mortalidade"]
Grafico_de_barras_por = "Obitos por dia" #@param ["Obitos por dia", "Novos Confirmados por dia", "Casos Confirmados por 100k habitantes", "Total de Confirmados",  "Total de Óbitos", "Taxa de Mortalidade"]

In [3]:

#baixar csv do dataset
headers = {
    'Content-Type': 'application/json;charset=UTF-8', 'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7'
}
url="https://brasil.io/dataset/covid19/caso_full/?state="+Estado+"&is_last=true&place_type=city&format=csv"
response = requests.get(url, headers=headers)
file_object = io.StringIO(response.content.decode('utf-8'))
covid = pd.read_csv(file_object)

url="https://brasil.io/dataset/covid19/caso_full/?state="+Estado+"&place_type=state&format=csv"
response = requests.get(url, headers=headers)
file_object = io.StringIO(response.content.decode('utf-8'))
estado = pd.read_csv(file_object)

covid = covid.rename(columns={"last_available_confirmed_per_100k_inhabitants": "Casos Confirmados por 100k habitantes", "last_available_confirmed": "Total de Confirmados", "last_available_deaths": "Total de Obitos"
                    , "estimated_population_2019": "População Estimada em 2019", "city" : "Nome", "new_deaths": "Obitos por dia", "last_available_death_rate": "Taxa de Mortalidade", "new_confirmed": "Novos Confirmados por dia"})

estado = estado.rename(columns={"last_available_confirmed_per_100k_inhabitants": "Casos Confirmados por 100k habitantes", "last_available_confirmed": "Total de Confirmados", "last_available_deaths": "Total de Obitos"
                    , "estimated_population_2019": "População Estimada em 2019", "city" : "Nome", "new_deaths": "Obitos por dia", "last_available_death_rate": "Taxa de Mortalidade", "new_confirmed": "Novos Confirmados por dia"})
#remover nulos
covid = covid[covid['Casos Confirmados por 100k habitantes'] > 0]
idEstado = int(estado['city_ibge_code'].iloc[0])

In [4]:
#trazer json de malha geografica da api do Ibge


cidades = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/"+str(idEstado)+"/?formato=application/vnd.geo+json&resolucao=5", headers=headers)
cidades = cidades.json()

In [5]:
covid['codarea'] = covid['city_ibge_code']
covid = covid.set_index('codarea')
covid['codarea'] = covid.index.astype(int).astype(str)

In [6]:
covid = covid.sort_values(by=[top_10_cidades_em], ascending=False)
covid[['Nome', 'Casos Confirmados por 100k habitantes', 'Total de Confirmados', 'Total de Obitos', 'Taxa de Mortalidade', 'População Estimada em 2019']].head(10)
#.style.set_caption('10 cidades com maior '+Mapa)

,Nome,Casos Confirmados por 100k habitantes,Total de Confirmados,Total de Obitos,Taxa de Mortalidade,População Estimada em 2019
codarea,,,,,,
3512605.0,Coronel Macedo,21.36296,1,1,1.0000,4681.0
3550001.0,São Luiz do Paraitinga,149.71461,16,5,0.3125,10687.0
3501400.0,Álvaro de Carvalho,191.31433,10,3,0.3000,5227.0
3503158.0,Arapeí,162.00891,4,1,0.2500,2469.0
3516507.0,Gabriel Monteiro,864.55331,24,4,0.1667,2776.0
3543238.0,Ribeirão dos Índios,584.26966,13,2,0.1538,2225.0
3516804.0,Gastão Vidigal,1643.09484,79,12,0.1519,4808.0
3554656.0,Torre de Pedra,580.43118,14,2,0.1429,2412.0
3544608.0,Sabino,590.33989,33,4,0.1212,5590.0


In [7]:
estado = estado.sort_values(by=['date'])
estado = estado.tail(90)

In [8]:
estado = estado.set_index('date')
estado['date'] = estado.index
estado['Media 7 dias'] = estado[Grafico_de_barras_por].rolling(window=7).mean()

In [9]:
from plotly.graph_objs.layout.shape import Line
fig = go.Figure(
    data=[go.Bar(y=estado[Grafico_de_barras_por], x=estado.index, name=Grafico_de_barras_por)
    , go.Line(y=estado['Media 7 dias'], x=estado.index, name="Media 7 dias")
    ],
    layout_title_text=Grafico_de_barras_por+" com Média de 7 dias"
)
fig.show()

In [10]:
#adicionar propriedades no geojson
maxLatitude = -200
maxLongitude = -200
minLatitude = 200
minLongitude = 200
#adicionar campos para layout no mapa para cada cidade
for cidade in cidades['features']: 
  codarea = int(cidade['properties']['codarea'])
  if codarea in covid.index.values:
    cidade['properties']['Cidade'] = str(covid.loc[codarea, "Nome"])
    cidade['properties']['Data Informacao'] = covid.loc[codarea, "date"]
    cidade['properties']['Populacao'] = int(covid.loc[codarea, "População Estimada em 2019"])
    cidade['properties']['Casos por 100k Habitantes'] = int(covid.loc[codarea, "Casos Confirmados por 100k habitantes"])
    cidade['properties']['Total de Confirmados'] = int(covid.loc[codarea, "Total de Confirmados"])
    cidade['properties']['Total de Obitos'] = int(covid.loc[codarea, "Total de Obitos"])
    cidade['properties']['Taxa de mortalidade'] = float(covid.loc[codarea, "Taxa de Mortalidade"])
  else:
    cidade['properties']['Cidade'] = 'Sem Dados'
    cidade['properties']['Data Informacao'] = ''
    cidade['properties']['Populacao'] = 0
    cidade['properties']['Casos por 100k Habitantes'] = 0
    cidade['properties']['Total de Confirmados'] = 0
    cidade['properties']['Total de Obitos'] = 0
    cidade['properties']['Taxa de mortalidade'] = 0
  maxLongitude = max(cidade['properties']['centroide'][0], maxLongitude)
  maxLatitude = max(cidade['properties']['centroide'][1], maxLatitude)
  minLongitude = min(cidade['properties']['centroide'][0], minLongitude)
  minLatitude = min(cidade['properties']['centroide'][1], minLatitude)

In [11]:
#mapa com casos por cidade
m = folium.Map(
    location=[(maxLatitude + minLatitude)/2, (maxLongitude + minLongitude)/2]
)

m.fit_bounds([[minLatitude, minLongitude], [maxLatitude, maxLongitude]]) 

c = folium.Choropleth(
    geo_data = cidades, 
    data = covid,
    columns = ['codarea', Mapa_coropletico_por],
    key_on = 'feature.properties.codarea',
    bins = 4,
    fill_color = 'YlOrRd', 
    fill_opacity = 0.7, 
    line_opacity = 0.3,
    legend_name = Mapa_coropletico_por
).add_to(m)

c.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=['Cidade', 'Data Informacao', 'Casos por 100k Habitantes', 'Populacao', 'Total de Confirmados', 'Total de Obitos', 'Taxa de mortalidade'], labels=True)
)

folium.LayerControl().add_to(m)

display(m)